In [ ]:
import torch as th
import numpy as np


In [ ]:
a = th.ones((2, 3, 4))
th.sum(a, (0))

In [ ]:
a = np.ones((2, 3, 4))
np.sum(a, (0))

In [ ]:
shape = (268435466, 2)
a = th.ones(shape).cuda().to(th.int64)

print("// start reduction")
b = th.sum(a, 1, keepdim=True)
print("// end reduction")
print(b.shape)
print(b)

In [ ]:
b = th.sum(a, 1, keepdim=True)
print(b.shape)
print(b)

# [Before]
# dim_to_split: 1
# shape: (2 268435466)
# [After]
# this->shape: (2 134217733)
# copy->shape: (2 134217733)
# [Before]
# dim_to_split: 1
# shape: (2 134217733)
# [After]
# this->shape: (2 67108867)
# copy->shape: (2 67108866)
# [Before]
# dim_to_split: 1
# shape: (2 134217733)
# [After]
# this->shape: (2 67108867)
# copy->shape: (2 67108866)

# Open3D
# [Before]
# dim_to_split: 1
# Shape: (2, 268435466)
# [After]
# this->shape: (2, 134217733)
# copy->shape: (2, 134217733)
# [Before]
# dim_to_split: 1
# Shape: (2, 134217733)
# [After]
# this->shape: (2, 67108867)
# copy->shape: (2, 67108866)
# [Before]
# dim_to_split: 1
# Shape: (2, 134217733)
# [After]
# this->shape: (2, 67108867)
# copy->shape: (2, 67108866)

In [ ]:
# shape = (100, 2)
# a = th.ones(shape).cuda().to(th.int64)
# b = th.sum(a, 1, keepdim=True)
# print(b.shape)
# print(b)

In [ ]:
# shape = (2, 268435466)
# a = th.ones(shape).cuda().to(th.int64)
# b = th.sum(a, 1, keepdim=True)
# print(b.shape)
# print(b)
# [Before]
# dim_to_split: 1
# this->ndim(): 2
# shape: 268435466, 2
# [After]
# this->ndim(): 1
# this->shape(): 268435466, 1
# copy->ndim(): 1
# copy->shape(): 268435466, 1

In [ ]:
shape = (2, 268435466)
a = th.ones(shape).cuda()
b = th.sum(a, 0, keepdim=True)
print(b.shape)
print(b)
# [Before]
# dim_to_split: 0
# this->ndim(): 2
# shape: 2, 268435466
# [After]
# this->ndim(): 1
# this->shape(): 268435466, 268435466
# copy->ndim(): 1
# copy->shape(): 268435466, 268435466

In [ ]:
shape = (32767, 2)
shape = (268435466, 2)
a = th.ones(shape).cuda()

b = th.sum(a, 0)
print(b.shape)
print(b)

b = th.sum(a, 1)
print(b.shape)
print(b)

In [ ]:
a = th.ones(shape)

b = th.sum(a, 0)
print(b.shape)
print(b)

b = th.sum(a, 1)
print(b.shape)
print(b)

In [ ]:
a = np.ones(shape)

b = np.sum(a, 0)
print(b.shape)
print(b)

b = np.sum(a, 1)
print(b.shape)
print(b)

In [ ]:
shape = (268435466, 2)
a = th.ones(shape)

b = th.sum(a, 0)
print(b.shape)
print(b)

b = th.sum(a, 1)
print(b.shape)
print(b)

In [ ]:
import numpy as np

a = np.ones((2, 3))
b = np.sum(a, (0))
print(b.shape)

In [ ]:
a = np.ones((2, 3, 1))
b = np.zeros((3, 4))

np.sum(a, axis=(0), out=b)
print(b.shape)

c = np.sum(a, axis=(0))
print(c.shape)

In [ ]:
import numpy as np

def gen_reduce_test_code(np_op, o3d_op_name, keepdims=(False,)):

    a = np.array([22, 23, 20, 9, 6, 14, 18, 13, 15, 3, 17, 0, 7, 21, 11, 1, 4, 2, 10, 19, 5, 8, 16, 12])
    shape = (2, 3, 4)
    a = a.reshape(shape)
    dims = [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]

    def list_to_str(l, is_float=True):
        if is_float:
            return ", ".join([str(v) + ".f" for v in l])
        else:
            return ", ".join([str(v) for v in l])

    src_str = list_to_str(a.flatten())
    shape_str = list_to_str(shape, False)

    for keepdim in keepdims:
        print(f"//////////////")
        print(f"// [{o3d_op_name}, keepdim={keepdim}]")
        
        print("Device device = GetParam();")
        print(f"Tensor src(std::vector<float>({{{src_str}}}), {{{shape_str}}}, Dtype::Float32, device);")
        print("Tensor dst;")
        print("")
        for dim in dims:
            b = np_op(a, axis=dim, keepdims=keepdim)
            dim_str = list_to_str(dim, False)
            reduced_shape_str = list_to_str(b.shape, False)
            keepdim_str = "true" if keepdim else "false"
            print(f"dst = src.{o3d_op_name}({{{dim_str}}}, {keepdim_str});")
            print(f"EXPECT_EQ(dst.GetShape(), SizeVector({{{reduced_shape_str}}}));")
            print(f"EXPECT_EQ(dst.ToFlatVector<float>(), std::vector<float>({{{list_to_str(b.flatten())}}}));")
            print("")
        print(f"//////////////")
            
    print("\n*****************************************************\n")

In [ ]:
gen_reduce_test_code(np.sum, "Sum", keepdims=(True, False))

In [ ]:
gen_reduce_test_code(np.prod, "Prod", keepdims=(False,))

In [ ]:
gen_reduce_test_code(np.min, "Min", keepdims=(False,))

In [ ]:
gen_reduce_test_code(np.max, "Max", keepdims=(False,))

In [ ]:
import numpy as np

def gen_arg_reduce_test_code(np_op, o3d_op_name):
    a = np.array([22, 23, 20, 9, 6, 14, 18, 13, 15, 3, 17, 0, 7, 21, 11, 1, 4, 2, 10, 19, 5, 8, 16, 12])
    shape = (2, 3, 4)
    a = a.reshape(shape)
    dims = [None, 0, 1, 2]

    def list_to_str(l):
        return ", ".join([str(v) for v in l])

    src_str = list_to_str(a.flatten())
    shape_str = list_to_str(shape)

    print(f"//////////////")
    print(f"// [{o3d_op_name}]")
    print("Device device = GetParam();")
    print(f"Tensor src(std::vector<float>({{{src_str}}}), {{{shape_str}}}, Dtype::Float32, device);")
    print("Tensor dst;")
    print("")
    for dim in dims:
        b = np_op(a, axis=dim)
        dim_str = "0, 1, 2" if dim is None else str(dim)
        reduced_shape_str = list_to_str(b.shape)
        print(f"dst = src.{o3d_op_name}({{{dim_str}}});")
        print(f"EXPECT_EQ(dst.GetShape(), SizeVector({{{reduced_shape_str}}}));")
        print(f"EXPECT_EQ(dst.ToFlatVector<int64_t>(), std::vector<int64_t>({{{list_to_str(b.flatten())}}}));")
        print("")
    print(f"//////////////")
    print("\n*****************************************************\n")
    
gen_arg_reduce_test_code(np.argmin, "ArgMin")
gen_arg_reduce_test_code(np.argmax, "ArgMax")

In [ ]:
a = np.array([])
np.argmin(a)